In [1]:
# Install Necessary Libraries
!pip install transformers pandas scikit-learn torch

In [2]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

In [3]:
# Load Train and Test Data
train_df = pd.read_csv('final_augmented_data (tamil).csv')
test_df = pd.read_csv('tam_test_data_hum_ai.csv')

In [4]:
# Drop unnecessary columns and rename columns
train_df = train_df.drop("back_translated_DATA", axis=1)
train_df = train_df.rename(columns={"back_translated_DATA_tamil": "DATA"})

In [5]:
# Ensure 'DATA' column is moved next to 'ID'
id_index = train_df.columns.get_loc("ID")
cols = list(train_df.columns)
cols.insert(id_index + 1, cols.pop(cols.index("DATA")))
train_df = train_df.loc[:, cols]

In [6]:
# Encode Labels
le = LabelEncoder()
train_df['LABEL'] = le.fit_transform(train_df['LABEL'])

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# **Transformer 1:**
https://huggingface.co/MayZhou/e5-small-lora-ai-generated-detector

In [8]:

tokenizer = AutoTokenizer.from_pretrained("MayZhou/e5-small-lora-ai-generated-detector")
model = AutoModelForSequenceClassification.from_pretrained("MayZhou/e5-small-lora-ai-generated-detector")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

# **Transformer 2**:
https://huggingface.co/idajikuu/AI-HUMAN-detector

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("idajikuu/AI-HUMAN-detector")
model = AutoModelForSequenceClassification.from_pretrained("idajikuu/AI-HUMAN-detector")

Some weights of the model checkpoint at idajikuu/AI-HUMAN-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# **Transformer 3**
https://huggingface.co/akshayvkt/detect-ai-text

In [18]:
# tokenizer = AutoTokenizer.from_pretrained("akshayvkt/detect-ai-text")
# model = AutoModelForSequenceClassification.from_pretrained("akshayvkt/detect-ai-text")

In [19]:
train_encodings = tokenizer(train_df['DATA'].tolist(), truncation=True, padding=True, max_length=512)
train_labels = train_df['LABEL'].tolist()

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

# Assuming train_encodings and train_labels are already defined

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels) # Create a Dataset object


In [21]:
# Training Arguments
training_args = TrainingArguments(
    output_dir='./model_output',
    evaluation_strategy="no",  # No evaluation during training
    num_train_epochs=4,
    per_device_train_batch_size=4,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=50,
    save_strategy="steps",
    save_total_limit=1
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
# Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)
trainer.train()

Step,Training Loss
10,0.099500
20,0.105100
30,0.114600
40,0.109100
50,0.356500
60,0.022400
70,0.246300
80,0.109900
90,0.296700
100,0.161100


TrainOutput(global_step=1616, training_loss=0.24177776082962898, metrics={'train_runtime': 139.677, 'train_samples_per_second': 46.278, 'train_steps_per_second': 11.57, 'total_flos': 22454366876928.0, 'train_loss': 0.24177776082962898, 'epoch': 4.0})

In [23]:
print(f"Missing values in test_df['DATA']: {test_df['DATA'].isna().sum()}")

Missing values in test_df['DATA']: 0


In [24]:
# Predict Labels for Test Data
# Check for missing values in 'DATA' column of test_df


# Remove rows with missing values in 'DATA' column
test_df.dropna(subset=['DATA'], inplace=True)
# Recheck if missing values were removed
print(f"Missing values in test_df['DATA'] after dropna: {test_df['DATA'].isna().sum()}")

Missing values in test_df['DATA'] after dropna: 0


In [25]:
# Tokenize the test data after removing any missing values.
test_encodings = tokenizer(test_df['DATA'].tolist(), truncation=True, padding=True, max_length=512)

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

test_dataset = TestDataset(test_encodings)

predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(axis=-1)

In [26]:
# Decode Predicted Labels
decoded_labels = le.inverse_transform(y_pred)

# Add Predictions to Test DataFrame
test_df['LABEL'] = decoded_labels

# **Using Transformer 1**

In [17]:
# prompt: compare the labels with the tamil-test.xlsx - Sheet1.csv file labels and calculate accuracy,recall,f1_score,precision

# Load the actual labels from the provided CSV file
actual_labels_df = pd.read_csv('tamil-test.xlsx - Sheet1.csv')

# The column name is 'Label', not 'LABEL'
actual_labels = actual_labels_df['Label'].values

# Ensure both predicted and actual labels have the same length
min_len = min(len(decoded_labels), len(actual_labels))
decoded_labels = decoded_labels[:min_len]
actual_labels = actual_labels[:min_len]


from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

# Calculate metrics
accuracy = accuracy_score(actual_labels, decoded_labels)
recall = recall_score(actual_labels, decoded_labels, average='weighted') # Use weighted average for multiclass
f1 = f1_score(actual_labels, decoded_labels, average='weighted') # Use weighted average for multiclass
precision = precision_score(actual_labels, decoded_labels, average='weighted') # Use weighted average for multiclass


print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")

Accuracy: 0.62
Recall: 0.62
F1 Score: 0.5922916666666667
Precision: 0.6478947368421052


# **Using Transformer 2:**

In [ ]:
# prompt: compare the labels with the tamil-test.xlsx - Sheet1.csv file labels and calculate accuracy,recall,f1_score,precision

# Load the actual labels from the provided CSV file
actual_labels_df = pd.read_csv('tamil-test.xlsx - Sheet1.csv')

# The column name is 'Label', not 'LABEL'
actual_labels = actual_labels_df['Label'].values

# Ensure both predicted and actual labels have the same length
min_len = min(len(decoded_labels), len(actual_labels))
decoded_labels = decoded_labels[:min_len]
actual_labels = actual_labels[:min_len]


from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

# Calculate metrics
accuracy = accuracy_score(actual_labels, decoded_labels)
recall = recall_score(actual_labels, decoded_labels, average='weighted') # Use weighted average for multiclass
f1 = f1_score(actual_labels, decoded_labels, average='weighted') # Use weighted average for multiclass
precision = precision_score(actual_labels, decoded_labels, average='weighted') # Use weighted average for multiclass


print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")

Accuracy: 0.7
Recall: 0.7
F1 Score: 0.6970779220779221
Precision: 0.7033333333333333


# **Using transformer 3**

In [27]:
# prompt: compare the labels with the tamil-test.xlsx - Sheet1.csv file labels and calculate accuracy,recall,f1_score,precision

# Load the actual labels from the provided CSV file
actual_labels_df = pd.read_csv('tamil-test.xlsx - Sheet1.csv')

# The column name is 'Label', not 'LABEL'
actual_labels = actual_labels_df['Label'].values

# Ensure both predicted and actual labels have the same length
min_len = min(len(decoded_labels), len(actual_labels))
decoded_labels = decoded_labels[:min_len]
actual_labels = actual_labels[:min_len]


from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

# Calculate metrics
accuracy = accuracy_score(actual_labels, decoded_labels)
recall = recall_score(actual_labels, decoded_labels, average='weighted') # Use weighted average for multiclass
f1 = f1_score(actual_labels, decoded_labels, average='weighted') # Use weighted average for multiclass
precision = precision_score(actual_labels, decoded_labels, average='weighted') # Use weighted average for multiclass


print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")

Accuracy: 0.48
Recall: 0.48
F1 Score: 0.33729729729729735
Precision: 0.26


# **Save Predictions**

In [ ]:

test_df.to_csv('Team Absolute Zero_tamil.tsv', sep='\t', index=False)

print("Predictions saved to 'test_predictions.tsv'")